##  Covid-19 in Seoul
* http://www.seoul.go.kr/coronaV/coronaStatus.do

In [352]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from fake_useragent import UserAgent
import re
import time
from selenium.webdriver.firefox.options import Options

In [353]:
target_url = 'http://www.seoul.go.kr/coronaV/coronaStatus.do'
browser_local_url = r'C:/Program Files/Mozilla Firefox/firefox.exe'
selenium_driver_local_url = r'./driver/geckodriver.exe'

In [354]:
capabilities = webdriver.DesiredCapabilities().FIREFOX
capabilities["marionette"] = True
binary = FirefoxBinary(browser_local_url)
driver = webdriver.Firefox(firefox_binary = binary, 
                           capabilities = capabilities, 
                           executable_path = selenium_driver_local_url)

In [355]:
driver.get(target_url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [356]:
page_nums = soup.findAll("a", {'class':'paginate_button'})

In [357]:
page_num_list = []
for i in page_nums:
    try:
        temp_page = int(i.get_text())
        if not temp_page in page_num_list:
            page_num_list.append(temp_page)
    except:
        continue
end_of_page_num = max(page_num_list)
print("End of the page num: %d" %(end_of_page_num))

End of the page num: 17


In [358]:
df = pd.DataFrame(columns = ['seoul_patient_num', 'national_patient_num', 'day_of_detection', 'address', 'travel', 'exposure', 'treatment'])
df

,seoul_patient_num,national_patient_num,day_of_detection,address,travel,exposure,treatment


In [359]:
row_index = 0
click_next_page = 4
click_crrt_page = 3
driver.get(target_url)
for page_num in range(1, end_of_page_num+1):
    ## 1 ~ 5 page
    if page_num <= 5:
        print("Crawling at webpage %d" %(page_num))
        driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(page_num)  +']').click()
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    ## 6page ~ end of page -2
    elif (page_num > 5) & (page_num <= end_of_page_num - 2):
        print("Crawling at webpage %d" %(page_num))
        while True:
            driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(click_crrt_page)  +']').click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser') 
            crr_page_num = soup.findAll("a", {'class':'paginate_button'})
            crr_page_list = []
            for i in crr_page_num:
                try:
                    crr_page_list.append(int(i.get_text()))
                except:
                    pass
            if (page_num in crr_page_list) and (crr_page_list.index(page_num) == 3):
                driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(click_crrt_page)  +']').click()
                break                                
            else:
                #click next page
                driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(click_next_page)  +']').click()
         
        # end of page - 1
    elif page_num <= end_of_page_num - 1:
        print("Crawling at webpage %d" %(page_num))
        while True:
            driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(click_crrt_page)  +']').click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser') 
            crr_page_num = soup.findAll("a", {'class':'paginate_button'})
            crr_page_list = []
            for i in crr_page_num:
                try:
                    crr_page_list.append(int(i.get_text()))
                except:
                    pass
            if (page_num in crr_page_list) and (crr_page_list.index(page_num) == 4):
                driver.find_element_by_xpath(' //*[@id="DataTables_Table_0_paginate"]/span/a[5]').click()
                break                                
            else:
                #click next page
                driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(click_next_page)  +']').click()  
                
  
    # end of page
    else:
        print("Crawling at webpage %d" %(page_num))
        while True:
            driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(click_crrt_page)  +']').click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser') 
            crr_page_num = soup.findAll("a", {'class':'paginate_button'})
            crr_page_list = []
            for i in crr_page_num:
                try:
                    crr_page_list.append(int(i.get_text()))
                except:
                    pass
            if (page_num in crr_page_list) and (crr_page_list.index(page_num) == 5):
                driver.find_element_by_xpath(' //*[@id="DataTables_Table_0_paginate"]/span/a[6]').click()
                break                                
            else:
                #click next page
                driver.find_element_by_xpath('//*[@id="DataTables_Table_0_paginate"]/span/a[' + str(click_next_page)  +']').click()            
                
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')                
    parsed_soup = soup.findAll('table')[6]
    for i in parsed_soup.findAll('tr'):
        col_cnt = 0
        if len(i.findAll(['p', 'td'])) == 0:
            continue
        for col_cnt, j in enumerate(i.findAll(['p', 'td'])):
            if col_cnt == 0:
                df.loc[row_index, 'seoul_patient_num'] = j.get_text()
            elif col_cnt == 1:
                df.loc[row_index, 'national_patient_num'] = j.get_text()
            elif col_cnt == 2:
                df.loc[row_index, 'day_of_detection'] = j.get_text()
            elif col_cnt == 3:
                df.loc[row_index, 'address'] = j.get_text()
            elif col_cnt == 4:
                df.loc[row_index, 'travel'] = j.get_text()
            elif col_cnt == 5:
                df.loc[row_index, 'exposure'] = j.get_text()
            else:
                df.loc[row_index, 'treatment'] = j.get_text()
        row_index = row_index + 1  


Crawling at webpage 1
Crawling at webpage 2
Crawling at webpage 3
Crawling at webpage 4
Crawling at webpage 5
Crawling at webpage 6
Crawling at webpage 7
Crawling at webpage 8
Crawling at webpage 9
Crawling at webpage 10
Crawling at webpage 11
Crawling at webpage 12
Crawling at webpage 13
Crawling at webpage 14
Crawling at webpage 15
Crawling at webpage 16
Crawling at webpage 17


In [351]:
df

,seoul_patient_num,national_patient_num,day_of_detection,address,travel,exposure,treatment
0,1612,14388,8.2.,동대문구,-,타시도 확진자 접촉,서울의료원
1,1611,14386,8.2.,대구,이라크,해외 접촉 추정,국립중앙의료원
2,1610,14383,8.2.,강서구,필리핀,해외 접촉 추정,서북병원
3,1609,14381,8.2.,성북구,-,확인 중,보라매병원
4,1608,14382,8.2.,성남시,일본,해외 접촉 추정,서울백병원
...,...,...,...,...,...,...,...
1607,5,9,1.31.,성북구,-,#5 접촉,서울의료원(퇴원)
1608,4,7,1.30.,마포구,중국 우한시,해외 접촉,서울의료원(퇴원)
1609,3,6,1.30.,종로구,-,#3 접촉,서울대학교병원(퇴원)
1610,2,5,1.30.,중랑구,중국 우한시,해외 접촉,서울의료원(퇴원)


In [360]:
last_day = df.loc[0, "day_of_detection"]
last_day = last_day.replace(".", "_")
last_day

'8_2_'

In [361]:
file_name = f"seoul_covid_{last_day}.csv"
file_name

'seoul_covid_8_2_.csv'

In [362]:
df.to_csv(file_name, index = False)

In [363]:
pd.read_csv(file_name)

,seoul_patient_num,national_patient_num,day_of_detection,address,travel,exposure,treatment
0,1612,14388,8.2.,동대문구,-,타시도 확진자 접촉,서울의료원
1,1611,14386,8.2.,대구,이라크,해외 접촉 추정,국립중앙의료원
2,1610,14383,8.2.,강서구,필리핀,해외 접촉 추정,서북병원
3,1609,14381,8.2.,성북구,-,확인 중,보라매병원
4,1608,14382,8.2.,성남시,일본,해외 접촉 추정,서울백병원
...,...,...,...,...,...,...,...
1607,5,9,1.31.,성북구,-,#5 접촉,서울의료원(퇴원)
1608,4,7,1.30.,마포구,중국 우한시,해외 접촉,서울의료원(퇴원)
1609,3,6,1.30.,종로구,-,#3 접촉,서울대학교병원(퇴원)
1610,2,5,1.30.,중랑구,중국 우한시,해외 접촉,서울의료원(퇴원)


In [364]:
df = df.sort_values(["seoul_patient_num"], ascending=False)

In [365]:
df.head()

,seoul_patient_num,national_patient_num,day_of_detection,address,travel,exposure,treatment
713,999,11817,6.08.,관악구,-,리치웨이 관련,서울의료원(퇴원)
714,998,11816,6.08.,중랑구,-,kb 생명보험 관련,서울의료원(퇴원)
715,997,11826,6.08.,서대문구,-,확인 중,세브란스병원(사망)
716,996,11852,6.05.,구로구,-,확인 중,서남병원(퇴원)
717,995,11812,6.07.,강동구,-,리치웨이 관련,서남병원(퇴원)


In [366]:
df.tail()

,seoul_patient_num,national_patient_num,day_of_detection,address,travel,exposure,treatment
711,1001,11821,6.08.,강서구,-,확인 중,순천향병원(퇴원)
712,1000,11851,6.08.,서초구,미국,해외 접촉 추정,서울의료원(퇴원)
1512,100,5592,3.3.,동대문구,-,동대문구 관련,서북병원(퇴원)
1602,10,21,2.5.,성북구,-,#6 접촉,서울대학교병원(퇴원)
1611,1,2,1.24.,강서구,중국 우한시,해외 접촉,국립중앙의료원(퇴원)
